In [1]:
import findspark
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/18 12:42:06 WARN Utils: Your hostname, Lanz-Lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlp2s0)
25/08/18 12:42:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/18 12:42:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/18 12:42:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/18 12:42:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                         'Time on App',
                                         'Time on Website',
                                         'Length of Membership',
                                         'Yearly Amount Spent']
                                        , outputCol='features')

In [16]:
output = assembler.transform(data)

In [17]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [19]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [20]:
final_data = output.select('features', 'Yearly Amount Spent')

In [21]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [22]:
train_data, test_data = final_data.randomSplit([0.7, 0.3] )

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean| 503.54454221448964|
| stddev|  80.73790913651298|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [25]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                146|
|   mean| 489.05651496826056|
| stddev|  75.03378021403151|
|    min|  282.4712457199145|
|    max|  689.2356997616951|
+-------+-------------------+



In [26]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [27]:
lr_model = lr.fit(train_data)

25/08/18 12:52:59 WARN Instrumentation: [d9b0337c] regParam is zero, which might cause numerical instability and overfitting.


In [30]:
test_results = lr_model.evaluate(test_data)

In [31]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-3.35376171278767...|
|-1.47792889038100...|
|4.547473508864641...|
|-2.61479726759716...|
|1.762145984685048...|
|-3.41060513164848...|
|-6.42330633127130...|
|5.115907697472721...|
|-2.38742359215393...|
|-1.70530256582424...|
|-1.81898940354585...|
|-9.09494701772928...|
|-1.13686837721616...|
|-2.72848410531878...|
|-7.38964445190504...|
|-1.59161572810262...|
|3.240074875066057...|
|1.477928890381008...|
|-2.72848410531878...|
|-1.70530256582424...|
+--------------------+
only showing top 20 rows


In [32]:
test_results.rootMeanSquaredError

2.6125321986076923e-12

In [33]:
test_results.r2

1.0

In [34]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [35]:
unlabeled_data = test_data.select('features')

In [36]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.4925366965402...|
|[30.5743636841713...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.3091926408918...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5761319713222...|
|[31.6253601348306...|
|[31.6739155032749...|
|[31.7656188210424...|
|[31.8209982016720...|
|[31.8512531286083...|
|[31.8627411090001...|
|[31.8854062999117...|
+--------------------+
only showing top 20 rows


In [37]:
predictions = lr_model.transform(unlabeled_data)

In [38]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|319.92886980319696|
|[30.4925366965402...|  282.471245719916|
|[30.5743636841713...| 442.0644137580652|
|[30.8794843441274...| 490.2065999848573|
|[30.9716756438877...|494.63860975689096|
|[31.0662181616375...|448.93329320767776|
|[31.1239743499119...| 486.9470538397722|
|[31.3091926408918...| 432.7207178399331|
|[31.4252268808548...| 530.7667186547643|
|[31.4459724827577...|484.87696493512874|
|[31.5147378578019...|489.81248799646323|
|[31.5257524169682...| 443.9656268098828|
|[31.5761319713222...| 541.2265839893295|
|[31.6253601348306...| 376.3369007569269|
|[31.6739155032749...| 475.7250679098886|
|[31.7656188210424...| 496.5540816356087|
|[31.8209982016720...| 424.6752810132101|
|[31.8512531286083...| 472.9922466667969|
|[31.8627411090001...| 556.2981411740494|
|[31.8854062999117...| 390.1032729724772|
+--------------------+------------